# Basic Tensor Functionality
Notes based on http://deeplearning.net/software/theano/library/tensor/basic.html#libdoc-basic-tensor.

In [43]:
import numpy as np
import theano.tensor as T
from theano import function
import theano

Theano focuses on symbolic matrix expressions like the following:

In [44]:
x = T.fmatrix()

### Creation
All of the following expressions produce a 0-dimensional array of integers named _myvar_: 

In [45]:
x = T.scalar('myvar', dtype='int32')
x = T.iscalar('myvar')
x = T.TensorType(dtype='int32', broadcastable=())('myvar')

Available types are __scalar__ (0-d array), __vector__ (1-d array), __matrix__ (2-d array), __row__ (2-d array, 1 row), __col__ (2-d array, 1 column), __tensor3__ (3-d array), and __tensor4__ (4-d array). They are prefixed in each case with __b__ (byte), __w__ (16-bit int), __i__ (32-bit int), __l__ (64-bit int), __f__ (float), __d__ (double), and __c__ (complex). All of them accept an optional _name_ argument.

### Plural constructors
Suffixing one of the above constructors with an __s__ makes them return more variables:

In [46]:
x, y, z = T.dmatrices('x', 'y', 'z')

### Custom tensor types
New tensor types with non-standard broadcasting pattern can be created as seen below:

In [49]:
dtensor5 = T.TensorType('float64', (False,)*5)
x = dtensor5('x')

### Converting from Python objects
We can also create TensorVariables by calling __shared()__, which creates a variable whose value may be shared between multiple functions, i.e. the argument to __shared()__ _will not be copied_. Changes can be inspected using __get_value()__.

In [41]:
x = theano.shared(np.random.randn(3, 4))
x.get_value()

array([[ 0.27835781,  0.51699834,  0.04442313, -2.35251917],
       [ 1.90817437, -0.67738736,  0.15724621, -2.08492307],
       [-0.56873797, -0.38814212, -0.10803327,  0.35997669]])

Note: When using a number (anything _but_ a variable) in an arithmetic expression, Theano wraps it in a __TensorConstant__, since the result is always a __TensorVariable__.

In [50]:
x = T.dscalar('x')
y = x + 2
f = function([x], y)
f(2)

array(4.0)

__as_tensor_variable__ turns an argument $x$ into a TensorVariable or TensorConstant; used frequently to preprocess arguments and convert them to a TensorVariable or TensorConstant. In the below example, they are converted to TensorConstant.

In [56]:
x = T.as_tensor_variable([[1, 2], [2, 3]], 'x')
y = T.as_tensor_variable([[2, 3], [1, 2]], 'y')
z = x + y
f = function([], z)
f()

array([[3, 5],
       [3, 5]])

### TensorType and TensorVariable
#### broadcastable
__broadcastable__ is a tuple of True/False, one for each dimension. If broadcasting is False for a dimension, the length of that dimension can be 1 or more. If broadcasting is True, the length must be 1. A  1-d scalar has the pattern [True], a 2-d scalar [True, True], while a vector has [False], a matrix [False, False], and a row [True, False], etc.
When two arguments in an element-wise operation have a different number of dimensions, the pattern is __expanded to the left__ by padding with True. A vector's pattern thus becomes [True, False], i.e. a row, while a matrix would behave as a 1xNxP tensor ([True, False, False]).